In [8]:
import yfinance as yf
import pandas as pd
import numpy as np
import sys

def main():
    # Take stock ticker as input
    ticker_input = input("Enter the stock ticker: ")
    ticker = ticker_input.upper()

    # Get stock data
    stock = yf.Ticker(ticker)
    info = stock.info

    # Get financial statements
    income_stmt = stock.financials
    cash_flow = stock.cashflow
    balance_sheet = stock.balance_sheet

    # Check if data is available
    if income_stmt.empty or cash_flow.empty or balance_sheet.empty:
        print("Financial data not available for this ticker.")
        return  # Exit the function

    sector = info.get('sector', 'Unknown')
    company_name = info.get('shortName', 'Unknown')

    # Display available rows in cash_flow
    # print("Available rows in cash_flow DataFrame:")
    # print(cash_flow.index.tolist())

    # Define possible keys for operating cash flow and capital expenditures
    possible_operating_cf_keys = [
        'Total Cash From Operating Activities',
        'Net Cash Provided by Operating Activities',
        'Operating Cash Flow',
        'Cash from Operating Activities'
    ]

    possible_capex_keys = [
        'Capital Expenditures',
        'Investment in Property, Plant and Equipment',
        'Purchases of Property and Equipment',
        'Capital Expenditure'
    ]

    # Function to find available key
    def get_cash_flow_item(cash_flow_df, possible_keys):
        for key in possible_keys:
            if key in cash_flow_df.index:
                return cash_flow_df.loc[key]
        return None

    # Use the function to get data
    operating_cf = get_cash_flow_item(cash_flow, possible_operating_cf_keys)
    capex = get_cash_flow_item(cash_flow, possible_capex_keys)

    if operating_cf is None or capex is None:
        print("Necessary financial data not found in cash flow statement.")
        return  # Exit the function

    # Calculate Free Cash Flow (FCF)
    fcf = operating_cf - capex

    # Sort FCF by date
    fcf = fcf.sort_index(ascending=True)
    fcf_values = fcf.values
    years = fcf.index

    if len(fcf_values) < 2:
        print("Not enough data to perform DCF.")
        return  # Exit the function

    # Calculate historical growth rates
    growth_rates = []
    for i in range(1, len(fcf_values)):
        if fcf_values[i-1] != 0:
            growth = (fcf_values[i] - fcf_values[i-1]) / abs(fcf_values[i-1])
            growth_rates.append(growth)

    # Average growth rate
    if len(growth_rates) > 0:
        avg_growth_rate = np.mean(growth_rates)
    else:
        avg_growth_rate = 0.05  # Assume 5% growth rate if data insufficient

    # Calculate WACC
    # Cost of Equity using CAPM
    beta = info.get('beta', 1.0)
    risk_free_rate = 0.02  # 2% risk-free rate
    market_return = 0.08   # 8% expected market return
    cost_of_equity = risk_free_rate + beta * (market_return - risk_free_rate)

    # Cost of Debt
    try:
        total_debt = balance_sheet.loc['Short Long Term Debt'].iloc[0] + balance_sheet.loc['Long Term Debt'].iloc[0]
    except KeyError:
        total_debt = balance_sheet.loc['Long Term Debt'].iloc[0]
    try:
        interest_expense = income_stmt.loc['Interest Expense'].iloc[0]
    except KeyError:
        interest_expense = 0

    if total_debt > 0:
        cost_of_debt = abs(interest_expense) / total_debt
    else:
        cost_of_debt = 0

    # Tax Rate
    try:
        income_before_tax = income_stmt.loc['Income Before Tax'].iloc[0]
        income_tax_expense = income_stmt.loc['Income Tax Expense'].iloc[0]
        tax_rate = income_tax_expense / income_before_tax
    except KeyError:
        tax_rate = 0.21  # Assume 21% tax rate if data not available

    # Capital Structure
    market_cap = info.get('marketCap', 0)
    total_value = market_cap + total_debt
    if total_value == 0:
        print("Market capitalization or total debt data not available.")
        return

    weight_of_equity = market_cap / total_value
    weight_of_debt = total_debt / total_value

    # WACC Calculation
    WACC = (weight_of_equity * cost_of_equity) + (weight_of_debt * cost_of_debt * (1 - tax_rate))

    # Project FCF into the future
    forecast_years = 5
    forecast_fcf = []

    last_fcf = fcf_values[-1]

    for i in range(1, forecast_years + 1):
        projected_fcf = last_fcf * ((1 + avg_growth_rate) ** i)
        forecast_fcf.append(projected_fcf)

    # Discount forecasted FCF
    discounted_fcf = []
    for i in range(1, forecast_years + 1):
        pv = forecast_fcf[i - 1] / ((1 + WACC) ** i)
        discounted_fcf.append(pv)

    # Calculate terminal value
    terminal_growth_rate = 0.025
    terminal_value = forecast_fcf[-1] * (1 + terminal_growth_rate) / (WACC - terminal_growth_rate)
    terminal_value_pv = terminal_value / ((1 + WACC) ** forecast_years)

    # DCF value
    dcf_value = sum(discounted_fcf) + terminal_value_pv

    # Get shares outstanding
    shares_outstanding = info.get('sharesOutstanding', 0)

    if shares_outstanding == 0:
        print("Shares outstanding not available.")
        return  # Exit the function

    # Intrinsic value per share
    intrinsic_value_per_share = dcf_value / shares_outstanding

    # Prepare output
    data = {
        'Ticker': [ticker],
        'Company Name': [company_name],
        'Sector': [sector],
        'Intrinsic Value per Share': [intrinsic_value_per_share],
    }
    df = pd.DataFrame(data)

    # Output DataFrame and save to CSV
    print(df)
    # df.to_csv(f'{ticker}_valuation.csv', index=False)

if __name__ == "__main__":
    main()

  Ticker Company Name      Sector  Intrinsic Value per Share
0   AAPL   Apple Inc.  Technology                     184.19


In [7]:
import yfinance as yf
import pandas as pd
import numpy as np

def main():
    # Take stock ticker as input
    ticker_input = input("Enter the stock ticker: ")
    ticker = ticker_input.upper()

    # Get stock data
    stock = yf.Ticker(ticker)
    info = stock.info

    # Get financial statements
    income_stmt = stock.financials
    cash_flow = stock.cashflow
    balance_sheet = stock.balance_sheet

    # Check if data is available
    if income_stmt.empty or cash_flow.empty or balance_sheet.empty:
        print("Financial data not available for this ticker.")
        return  # Exit the function

    sector = info.get('sector', 'Unknown')
    company_name = info.get('shortName', 'Unknown')

    # Display available rows in cash_flow
    # print("\nAvailable rows in cash_flow DataFrame:")
    # print(cash_flow.index.tolist())

    # Define possible keys for operating cash flow and capital expenditures
    possible_operating_cf_keys = [
        'Total Cash From Operating Activities',
        'Net Cash Provided by Operating Activities',
        'Operating Cash Flow',
        'Cash from Operating Activities'
    ]

    possible_capex_keys = [
        'Capital Expenditures',
        'Investment in Property, Plant and Equipment',
        'Purchases of Property and Equipment',
        'Capital Expenditure'
    ]

    # Function to find available key
    def get_cash_flow_item(cash_flow_df, possible_keys):
        for key in possible_keys:
            if key in cash_flow_df.index:
                return cash_flow_df.loc[key]
        return None

    # Use the function to get data
    operating_cf = get_cash_flow_item(cash_flow, possible_operating_cf_keys)
    capex = get_cash_flow_item(cash_flow, possible_capex_keys)

    if operating_cf is None or capex is None:
        print("Necessary financial data not found in cash flow statement.")
        return  # Exit the function

    # Calculate Free Cash Flow (FCF)
    fcf = operating_cf - capex

    # Sort FCF by date
    fcf = fcf.sort_index(ascending=True)
    fcf_values = fcf.values
    years = fcf.index

    if len(fcf_values) < 2:
        print("Not enough data to perform DCF.")
        return  # Exit the function

    # Calculate historical growth rates
    growth_rates = []
    for i in range(1, len(fcf_values)):
        if fcf_values[i-1] != 0:
            growth = (fcf_values[i] - fcf_values[i-1]) / abs(fcf_values[i-1])
            growth_rates.append(growth)

    # Average growth rate
    if len(growth_rates) > 0:
        avg_growth_rate = np.mean(growth_rates)
    else:
        avg_growth_rate = 0.05  # Assume 5% growth rate if data insufficient

    # Calculate WACC
    # Cost of Equity using CAPM
    beta = info.get('beta', 1.0)
    risk_free_rate = 0.02  # 2% risk-free rate
    market_return = 0.08   # 8% expected market return
    cost_of_equity = risk_free_rate + beta * (market_return - risk_free_rate)

    # Cost of Debt
    try:
        total_debt = balance_sheet.loc['Short Long Term Debt'].iloc[0] + balance_sheet.loc['Long Term Debt'].iloc[0]
    except KeyError:
        try:
            total_debt = balance_sheet.loc['Long Term Debt'].iloc[0]
        except KeyError:
            total_debt = 0
    try:
        interest_expense = income_stmt.loc['Interest Expense'].iloc[0]
    except KeyError:
        interest_expense = 0

    if total_debt > 0 and interest_expense != 0:
        cost_of_debt = abs(interest_expense) / total_debt
    else:
        cost_of_debt = 0.03  # Assume 3% cost of debt if data not available

    # Tax Rate
    try:
        income_before_tax = income_stmt.loc['Income Before Tax'].iloc[0]
        income_tax_expense = income_stmt.loc['Income Tax Expense'].iloc[0]
        tax_rate = income_tax_expense / income_before_tax
    except KeyError:
        tax_rate = 0.21  # Assume 21% tax rate if data not available

    # Capital Structure
    market_cap = info.get('marketCap', 0)
    total_value = market_cap + total_debt
    if total_value == 0:
        print("Market capitalization or total debt data not available.")
        return

    weight_of_equity = market_cap / total_value
    weight_of_debt = total_debt / total_value

    # WACC Calculation
    WACC = (weight_of_equity * cost_of_equity) + (weight_of_debt * cost_of_debt * (1 - tax_rate))

    # Project FCF into the future
    forecast_years = 5
    forecast_fcf = []

    last_fcf = fcf_values[-1]

    for i in range(1, forecast_years + 1):
        projected_fcf = last_fcf * ((1 + avg_growth_rate) ** i)
        forecast_fcf.append(projected_fcf)

    # Discount forecasted FCF
    discounted_fcf = []
    for i in range(1, forecast_years + 1):
        pv = forecast_fcf[i - 1] / ((1 + WACC) ** i)
        discounted_fcf.append(pv)

    # Calculate terminal value
    terminal_growth_rate = 0.025
    terminal_value = forecast_fcf[-1] * (1 + terminal_growth_rate) / (WACC - terminal_growth_rate)
    terminal_value_pv = terminal_value / ((1 + WACC) ** forecast_years)

    # Enterprise Value (Total Company Valuation)
    enterprise_value = sum(discounted_fcf) + terminal_value_pv

    # Get Cash and Cash Equivalents
    try:
        cash_and_equiv = balance_sheet.loc['Cash'].iloc[0] + balance_sheet.loc['Short Term Investments'].iloc[0]
    except KeyError:
        try:
            cash_and_equiv = balance_sheet.loc['Cash'].iloc[0]
        except KeyError:
            cash_and_equiv = 0  # If cash data not available

    # Calculate Net Debt
    net_debt = total_debt - cash_and_equiv

    # Calculate Equity Value
    equity_value = enterprise_value - net_debt

    # Convert values to millions for readability
    enterprise_value_millions = enterprise_value / 1e6
    net_debt_millions = net_debt / 1e6
    equity_value_millions = equity_value / 1e6

    # Prepare output
    data = {
        'Ticker': [ticker],
        'Company Name': [company_name],
        'Sector': [sector],
        'Enterprise Value (Millions)': [enterprise_value_millions],
        'Net Debt (Millions)': [net_debt_millions],
        'Equity Value (Millions)': [equity_value_millions],
    }
    df = pd.DataFrame(data)

    # Format the output to show two decimal places
    pd.options.display.float_format = '{:,.2f}'.format

    # Output DataFrame and save to CSV

    
    print("\nValuation Results:")
    print(df)
    # df.to_csv(f'{ticker}_valuation.csv', index=False)

if __name__ == "__main__":
    main()


Valuation Results:
  Ticker Company Name      Sector  Enterprise Value (Millions)  \
0   ABBV  AbbVie Inc.  Healthcare                          NaN   

   Net Debt (Millions)  Equity Value (Millions)  
0            52,194.00                      NaN  


In [10]:
import yfinance as yf
import pandas as pd
import numpy as np

def main():
    # Take stock ticker as input
    ticker_input = input("Enter the stock ticker: ")
    ticker = ticker_input.upper()

    # Get stock data
    stock = yf.Ticker(ticker)
    info = stock.info

    # Get financial statements
    income_stmt = stock.financials
    cash_flow = stock.cashflow
    balance_sheet = stock.balance_sheet

    # Check if data is available
    if income_stmt.empty or cash_flow.empty or balance_sheet.empty:
        print("Financial data not available for this ticker.")
        return  # Exit the function

    sector = info.get('sector', 'Unknown')
    company_name = info.get('shortName', 'Unknown')

    # Display available rows in cash_flow
    # print("\nAvailable rows in cash_flow DataFrame:")
    # print(cash_flow.index.tolist())

    # Define possible keys for operating cash flow and capital expenditures
    possible_operating_cf_keys = [
        'Total Cash From Operating Activities',
        'Net Cash Provided by Operating Activities',
        'Operating Cash Flow',
        'Cash from Operating Activities'
    ]

    possible_capex_keys = [
        'Capital Expenditures',
        'Investment in Property, Plant and Equipment',
        'Purchases of Property and Equipment',
        'Capital Expenditure'
    ]

    # Function to find available key
    def get_cash_flow_item(cash_flow_df, possible_keys):
        for key in possible_keys:
            if key in cash_flow_df.index:
                return cash_flow_df.loc[key]
        return None

    # Use the function to get data
    operating_cf = get_cash_flow_item(cash_flow, possible_operating_cf_keys)
    capex = get_cash_flow_item(cash_flow, possible_capex_keys)

    if operating_cf is None or capex is None:
        print("Necessary financial data not found in cash flow statement.")
        return  # Exit the function

    # Calculate Free Cash Flow (FCF)
    fcf = operating_cf - capex

    # Check for zero or negative FCF values
    if fcf.isnull().values.any() or (fcf <= 0).all():
        print("FCF data is invalid or non-positive.")
        return

    # Sort FCF by date
    fcf = fcf.sort_index(ascending=True)
    fcf_values = fcf.values
    years = fcf.index

    if len(fcf_values) < 2:
        print("Not enough data to perform DCF.")
        return  # Exit the function

    # Calculate historical growth rates
    growth_rates = []
    for i in range(1, len(fcf_values)):
        if fcf_values[i-1] != 0:
            growth = (fcf_values[i] - fcf_values[i-1]) / abs(fcf_values[i-1])
            growth_rates.append(growth)

    # Average growth rate
    if len(growth_rates) > 0:
        avg_growth_rate = np.mean(growth_rates)
        if np.isnan(avg_growth_rate) or np.isinf(avg_growth_rate):
            avg_growth_rate = 0.05  # Default to 5% if invalid
    else:
        avg_growth_rate = 0.05  # Assume 5% growth rate if data insufficient

    # Calculate WACC
    # Cost of Equity using CAPM
    beta = info.get('beta', 1.0)
    if beta is None or np.isnan(beta):
        beta = 1.0  # Default beta
    risk_free_rate = 0.02  # 2% risk-free rate
    market_return = 0.08   # 8% expected market return
    cost_of_equity = risk_free_rate + beta * (market_return - risk_free_rate)

    # Ensure cost_of_equity is valid
    if np.isnan(cost_of_equity) or cost_of_equity <= 0:
        cost_of_equity = 0.08  # Default to 8%

    # Cost of Debt
    try:
        total_debt = balance_sheet.loc['Short Long Term Debt'].iloc[0] + balance_sheet.loc['Long Term Debt'].iloc[0]
    except KeyError:
        try:
            total_debt = balance_sheet.loc['Long Term Debt'].iloc[0]
        except KeyError:
            total_debt = 0
    try:
        interest_expense = income_stmt.loc['Interest Expense'].iloc[0]
    except KeyError:
        interest_expense = 0

    if total_debt > 0 and interest_expense != 0:
        cost_of_debt = abs(interest_expense) / total_debt
    else:
        cost_of_debt = 0.03  # Assume 3% cost of debt if data not available

    # Tax Rate
    try:
        income_before_tax = income_stmt.loc['Income Before Tax'].iloc[0]
        income_tax_expense = income_stmt.loc['Income Tax Expense'].iloc[0]
        if income_before_tax != 0:
            tax_rate = income_tax_expense / income_before_tax
        else:
            tax_rate = 0.21  # Default tax rate if division by zero
    except KeyError:
        tax_rate = 0.21  # Assume 21% tax rate if data not available

    if np.isnan(tax_rate) or tax_rate < 0 or tax_rate > 1:
        tax_rate = 0.21  # Default tax rate

    # Capital Structure
    market_cap = info.get('marketCap', 0)
    if market_cap is None or market_cap <= 0:
        print("Market capitalization data not available or invalid.")
        return

    total_value = market_cap + total_debt

    if total_value == 0:
        print("Total value is zero, cannot calculate weights.")
        return

    weight_of_equity = market_cap / total_value
    weight_of_debt = total_debt / total_value

    # WACC Calculation
    WACC = (weight_of_equity * cost_of_equity) + (weight_of_debt * cost_of_debt * (1 - tax_rate))

    if np.isnan(WACC) or WACC <= 0:
        print("WACC calculation invalid.")
        return

    # Ensure WACC is greater than terminal growth rate
    terminal_growth_rate = 0.025
    if WACC <= terminal_growth_rate:
        print("WACC is less than or equal to terminal growth rate. Adjusting WACC.")
        WACC = terminal_growth_rate + 0.01  # Set WACC slightly above terminal growth rate

    # Project FCF into the future
    forecast_years = 5
    forecast_fcf = []

    last_fcf = fcf_values[-1]

    for i in range(1, forecast_years + 1):
        projected_fcf = last_fcf * ((1 + avg_growth_rate) ** i)
        forecast_fcf.append(projected_fcf)

    # Discount forecasted FCF
    discounted_fcf = []
    for i in range(1, forecast_years + 1):
        pv = forecast_fcf[i - 1] / ((1 + WACC) ** i)
        discounted_fcf.append(pv)

    # Calculate terminal value
    terminal_value = forecast_fcf[-1] * (1 + terminal_growth_rate) / (WACC - terminal_growth_rate)
    terminal_value_pv = terminal_value / ((1 + WACC) ** forecast_years)

    # Check for NaN or infinite terminal value
    if np.isnan(terminal_value_pv) or np.isinf(terminal_value_pv):
        print("Terminal value calculation invalid.")
        return

    # Enterprise Value (Total Company Valuation)
    enterprise_value = sum(discounted_fcf) + terminal_value_pv

    # Check for NaN or infinite enterprise value
    if np.isnan(enterprise_value) or np.isinf(enterprise_value):
        print("Enterprise value calculation invalid.")
        return

    # Get Cash and Cash Equivalents
    try:
        cash_and_equiv = balance_sheet.loc['Cash'].iloc[0] + balance_sheet.loc['Short Term Investments'].iloc[0]
    except KeyError:
        try:
            cash_and_equiv = balance_sheet.loc['Cash'].iloc[0]
        except KeyError:
            cash_and_equiv = 0  # If cash data not available

    # Calculate Net Debt
    net_debt = total_debt - cash_and_equiv

    # Calculate Equity Value
    equity_value = enterprise_value - net_debt

    # Convert values to millions for readability
    enterprise_value_millions = enterprise_value / 1e6
    net_debt_millions = net_debt / 1e6
    equity_value_millions = equity_value / 1e6

    # Prepare output
    data = {
        'Ticker': [ticker],
        'Company Name': [company_name],
        'Sector': [sector],
        'Enterprise Value (Millions)': [enterprise_value_millions],
        'Net Debt (Millions)': [net_debt_millions],
        'Equity Value (Millions)': [equity_value_millions],
    }
    df = pd.DataFrame(data)

    # Format the output to show two decimal places
    pd.options.display.float_format = '{:,.2f}'.format

    # Output DataFrame and save to CSV
    print("\nValuation Results:")
    print(df)
    # df.to_csv(f'{ticker}_valuation.csv', index=False)

if __name__ == "__main__":
    main()

Enter the stock ticker:  intc


FCF data is invalid or non-positive.


In [1]:
import yfinance as yf
import pandas as pd
import numpy as np

def main():
    # Take stock ticker as input
    ticker_input = input("Enter the stock ticker: ")
    ticker = ticker_input.upper()

    # Get stock data
    stock = yf.Ticker(ticker)
    info = stock.info

    # Get financial statements
    income_stmt = stock.financials
    balance_sheet = stock.balance_sheet

    # Check if data is available
    if income_stmt.empty or balance_sheet.empty:
        print("Financial data not available for this ticker.")
        return  # Exit the function

    sector = info.get('sector', 'Unknown')
    company_name = info.get('shortName', 'Unknown')

    # Get EBITDA
    try:
        ebitda_series = income_stmt.loc['EBITDA']
        ebitda_series = ebitda_series.sort_index(ascending=True)
        ebitda_values = ebitda_series.values
    except KeyError:
        print("EBITDA data not available.")
        return  # Exit the function

    # Check for zero or negative EBITDA
    if ebitda_values[-1] <= 0 or np.isnan(ebitda_values[-1]):
        print("EBITDA data is invalid or non-positive.")
        return

    # Calculate historical EBITDA growth rates
    growth_rates = []
    for i in range(1, len(ebitda_values)):
        if ebitda_values[i-1] != 0:
            growth = (ebitda_values[i] - ebitda_values[i-1]) / abs(ebitda_values[i-1])
            growth_rates.append(growth)

    if len(growth_rates) > 0:
        avg_growth_rate = np.mean(growth_rates)
        if np.isnan(avg_growth_rate) or np.isinf(avg_growth_rate):
            avg_growth_rate = 0.05  # Default to 5% if invalid
    else:
        avg_growth_rate = 0.05  # Assume 5% growth rate if data insufficient

    # WACC Calculation
    # Cost of Equity using CAPM
    beta = info.get('beta', 1.0)
    if beta is None or np.isnan(beta):
        beta = 1.0  # Default beta
    risk_free_rate = 0.02  # 2% risk-free rate
    market_return = 0.08   # 8% expected market return
    cost_of_equity = risk_free_rate + beta * (market_return - risk_free_rate)

    # Ensure cost_of_equity is valid
    if np.isnan(cost_of_equity) or cost_of_equity <= 0:
        cost_of_equity = 0.08  # Default to 8%

    # Cost of Debt
    try:
        total_debt = balance_sheet.loc['Short Long Term Debt'].iloc[0] + balance_sheet.loc['Long Term Debt'].iloc[0]
    except KeyError:
        try:
            total_debt = balance_sheet.loc['Long Term Debt'].iloc[0]
        except KeyError:
            total_debt = 0
    try:
        interest_expense = income_stmt.loc['Interest Expense'].iloc[0]
    except KeyError:
        interest_expense = 0

    if total_debt > 0 and interest_expense != 0:
        cost_of_debt = abs(interest_expense) / total_debt
    else:
        cost_of_debt = 0.03  # Assume 3% cost of debt if data not available

    # Tax Rate
    try:
        income_before_tax = income_stmt.loc['Income Before Tax'].iloc[0]
        income_tax_expense = income_stmt.loc['Income Tax Expense'].iloc[0]
        if income_before_tax != 0:
            tax_rate = income_tax_expense / income_before_tax
        else:
            tax_rate = 0.21  # Default tax rate if division by zero
    except KeyError:
        tax_rate = 0.21  # Assume 21% tax rate if data not available

    if np.isnan(tax_rate) or tax_rate < 0 or tax_rate > 1:
        tax_rate = 0.21  # Default tax rate

    # Capital Structure
    market_cap = info.get('marketCap', 0)
    if market_cap is None or market_cap <= 0:
        print("Market capitalization data not available or invalid.")
        return

    total_value = market_cap + total_debt

    if total_value == 0:
        print("Total value is zero, cannot calculate weights.")
        return

    weight_of_equity = market_cap / total_value
    weight_of_debt = total_debt / total_value

    # WACC Calculation
    WACC = (weight_of_equity * cost_of_equity) + (weight_of_debt * cost_of_debt * (1 - tax_rate))

    if np.isnan(WACC) or WACC <= 0:
        print("WACC calculation invalid.")
        return

    # Ensure WACC is greater than terminal growth rate
    terminal_growth_rate = 0.025
    if WACC <= terminal_growth_rate:
        print("WACC is less than or equal to terminal growth rate. Adjusting WACC.")
        WACC = terminal_growth_rate + 0.01  # Set WACC slightly above terminal growth rate

    # Project EBITDA into the future using the average growth rate
    forecast_years = 5
    forecast_ebitda = []
    last_ebitda = ebitda_values[-1]

    for i in range(1, forecast_years + 1):
        projected_ebitda = last_ebitda * ((1 + avg_growth_rate) ** i)
        forecast_ebitda.append(projected_ebitda)

    # Assume constant D&A and calculate EBIT
    try:
        depreciation = income_stmt.loc['Depreciation'].iloc[0]
    except KeyError:
        depreciation = 0  # Assume zero if data not available

    forecast_ebit = [ebitda - depreciation for ebitda in forecast_ebitda]

    # Calculate NOPAT (Net Operating Profit After Tax)
    nopat = [ebit * (1 - tax_rate) for ebit in forecast_ebit]

    # Discount forecasted NOPAT
    discounted_cash_flows = []
    for i in range(1, forecast_years + 1):
        pv = nopat[i - 1] / ((1 + WACC) ** i)
        discounted_cash_flows.append(pv)

    # Calculate terminal value
    terminal_value = nopat[-1] * (1 + terminal_growth_rate) / (WACC - terminal_growth_rate)
    terminal_value_pv = terminal_value / ((1 + WACC) ** forecast_years)

    # Check for NaN or infinite terminal value
    if np.isnan(terminal_value_pv) or np.isinf(terminal_value_pv):
        print("Terminal value calculation invalid.")
        return

    # Enterprise Value
    enterprise_value = sum(discounted_cash_flows) + terminal_value_pv

    # Check for NaN or infinite enterprise value
    if np.isnan(enterprise_value) or np.isinf(enterprise_value):
        print("Enterprise value calculation invalid.")
        return

    # Get total debt (already calculated earlier)
    # total_debt variable is already defined

    # Get Cash and Cash Equivalents
    try:
        cash_and_equiv = balance_sheet.loc['Cash'].iloc[0] + balance_sheet.loc['Short Term Investments'].iloc[0]
    except KeyError:
        try:
            cash_and_equiv = balance_sheet.loc['Cash'].iloc[0]
        except KeyError:
            cash_and_equiv = 0  # If cash data not available

    # Calculate Net Debt
    net_debt = total_debt - cash_and_equiv

    # Calculate Equity Value
    equity_value = enterprise_value - net_debt

    # Convert values to millions
    enterprise_value_millions = enterprise_value / 1e6
    net_debt_millions = net_debt / 1e6
    equity_value_millions = equity_value / 1e6

    # Prepare output
    data = {
        'Ticker': [ticker],
        'Company Name': [company_name],
        'Sector': [sector],
        'Enterprise Value (Millions)': [enterprise_value_millions],
        'Net Debt (Millions)': [net_debt_millions],
        'Equity Value (Millions)': [equity_value_millions],
    }
    df = pd.DataFrame(data)

    # Format the output to show two decimal places
    pd.options.display.float_format = '{:,.2f}'.format

    # Output DataFrame and save to CSV
    print("\nValuation Results:")
    print(df)
    # df.to_csv(f'{ticker}_valuation.csv', index=False)

In [2]:
if __name__ == "__main__":
    main()

Enter the stock ticker:  msft



Valuation Results:
  Ticker           Company Name      Sector  Enterprise Value (Millions)  \
0   MSFT  Microsoft Corporation  Technology                 3,999,820.92   

   Net Debt (Millions)  Equity Value (Millions)  
0            42,688.00             3,957,132.92  
